# BeautyfulSoup 네이버 영화 리뷰 스크래핑

## 네이버 영화 리뷰 스크래핑

* 네이버 영화: https://movie.naver.com/movie/point/af/list.nhn

In [19]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_movie_reviews(mode, page_num):
  movie_review_df = pd.DataFrame(columns={"Title", "Score","Review"})
  url = "https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=" + str(mode) + "&target=after"
  idx =0
  for _ in range(0, page_num):
    movie_page = urllib.request.urlopen(url).read()
    movie_page_soup = BeautifulSoup(movie_page, 'html.parser')

    review_list = movie_page_soup.find_all('td', {'class':'title'})
    for review in review_list:
      title = review.find('a', {'class':'movie color_b'}).get_text()
      score = review.find('em').get_text()
      review_text = review.find('a', {'class':'report'}).get('href').split(',')[2]
      movie_review_df.loc[idx] = [title, score, review_text]
      idx +=1
      print("#", end="")
    try:
      url = "https://movie.naver.com" + movie_page_soup.find('a', {'class':'pg_next'}).get('href')
    except:
      break
  return movie_review_df



In [20]:
movie_review_df = get_movie_reviews(18847, 10)
movie_review_df

####################################################################################################

,Score,Title,Review
0,타이타닉,10,'마음 단단히 부여잡고 봐라 아무리 보고싶어도 꾹 참고 영화관에서 재상영할때 집중...
1,타이타닉,10,'왜 이걸 이제서야 본 것일까'
2,타이타닉,10,'첫 시청의 여운을 20살에 느낄수있어 감사합니다'
3,타이타닉,10,'다시 극장에서 보고싶은 영화'
4,타이타닉,10,''
...,...,...,...
95,타이타닉,10,'와 진짜 감동적인 스토리와 그에 걸맞는 주인공들
96,타이타닉,10,'인생 영화. 최고의 작품'
97,타이타닉,10,'여운이 짙게 남는 영화3시간이 넘는 러닝타임에도 금방 끝나버렸다'
98,타이타닉,10,'너무 좋은 영화지만 마음이 너무 아파서 두번은 못 볼꺼같은 영화


## 현재상영작 영화 리뷰 스크래핑

In [25]:
url = 'https://movie.naver.com/movie/point/af/list.nhn'
naver_movie = urllib.request.urlopen(url).read()
soup = BeautifulSoup(naver_movie, 'html.parser')

select = soup.find('select', {'id':'current_movie'})
movies = select.find_all('option')

movie_dict = {}
for movie in movies[1:]:
  movie_dict[movie.get('value')] = movie.get_text()

movie_review_df = pd.DataFrame(columns=("Title", "Score", "Review"))
for mcode in movie_dict:
  df = get_movie_reviews(mcode, 1)
  movie_review_df = pd.concat([movie_review_df, df])

movie_review_df



########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

,Title,Score,Review
0,7,테넷,'크리스토퍼 놀란 감독 작품 아니었으면 평점 6점 맞고 IPTV 개봉 했을 영화'
1,10,테넷,'인셉션이 쉽게 느껴지게 만드는 영화~~ㅋㅋ '
2,10,테넷,'진심이세상영화가아님 인셉션보다 0.0001못함'
3,10,테넷,'이해가 안되는데 재밌다고?라고 생각하겠지만 영화보면 안다'
4,10,테넷,'한번 더 봐야겠어 -_ -;;← 나만 이 생각하는 거 아니겠지?ㅜ'
...,...,...,...
5,9,왕중왕,'예수를 다룬 영화중에 가장 뛰어나다...'
6,2,왕중왕,'예수 같은 왕 둔 적 없다.'
7,10,왕중왕,'예수님의 왕 자리는 대통령 이나 수상이나 황제나 천왕 의 자리가 아니...
8,10,왕중왕,'20세기가 낳은 최고의 명작'
